# Chapter 3 - Classification with k-nearest neighbors

## Listing 3.1

In [1]:
install_load <- function (package1, ...)  {   

   # convert arguments to vector
   packages <- c(package1, ...)

   # start loop to determine if each package is installed
   for(package in packages){

       # if package is installed locally, load
       if(package %in% rownames(installed.packages()))
          do.call('library', list(package))

       # if package is not installed locally, download, then load
       else {
          install.packages(package)
          do.call("library", list(package))
       }
   } 
}

install_load('mlr', 'mclust', 'tidyverse')

data(diabetes, package="mclust")
diabetesTib <- as_tibble(diabetes)

summary(diabetesTib)
diabetesTib

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
also installing the dependencies ‘ParamHelpers’, ‘BBmisc’, ‘data.table’, ‘parallelMap’

Loading required package: ParamHelpers
'mlr' is in maintenance mode since July 2019. Future development
efforts will go into its successor 'mlr3' (<https://mlr3.mlr-org.com>).
Package 'mclust' version 5.4.3
Type 'citation("mclust")' for citing this R package in publications.
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::map()    masks mclust::map()


      class       glucose       insulin            sspg      
 Chemical:36   Min.   : 70   Min.   :  45.0   Min.   : 10.0  
 Normal  :76   1st Qu.: 90   1st Qu.: 352.0   1st Qu.:118.0  
 Overt   :33   Median : 97   Median : 403.0   Median :156.0  
               Mean   :122   Mean   : 540.8   Mean   :186.1  
               3rd Qu.:112   3rd Qu.: 558.0   3rd Qu.:221.0  
               Max.   :353   Max.   :1568.0   Max.   :748.0  

class,glucose,insulin,sspg
<fct>,<dbl>,<dbl>,<dbl>
Normal,80,356,124
Normal,97,289,117
Normal,105,319,143
Normal,90,356,199
Normal,90,323,240
Normal,86,381,157
Normal,100,350,221
Normal,85,301,186
Normal,97,379,142


## Listing 3.2

In [2]:
ggplot(diabetesTib, aes(glucose, insulin, col = class)) + 
    geom_point() +
    theme_bw()

ggplot(diabetesTib, aes(sspg, insulin, col = class)) +
    geom_point() +
    theme_bw()

ggplot(diabetesTib, aes(sspg, glucose, col = class)) +
    geom_point() +
    theme_bw()

## Exercise 1

In [3]:
ggplot(diabetesTib, aes(glucose, insulin, shape = class)) + 
    geom_point() +
    theme_bw()

ggplot(diabetesTib, aes(glucose, insulin, col = class, shape = class)) + 
    geom_point() +
    theme_bw()

## Section 3.2.3

In [4]:
diabetesTask <- makeClassifTask(data = diabetesTib, target = "class")
print(diabetesTask)

knn <- makeLearner("classif.knn", par.vals = list ("k" = 2))
knnModel <- train(knn, diabetesTask)
knnPredictions <- predict(knnModel, newdata = diabetesTib)

knnPerf <- performance(knnPredictions, measures = list(mmce, acc))
knnPerf

Warning message in makeTask(type = type, data = data, weights = weights, blocking = blocking, :
“Provided data is not a pure data.frame but from class tbl_df, hence it will be converted.”

Supervised task: diabetesTib
Type: classif
Target: class
Observations: 145
Features:
   numerics     factors     ordered functionals 
          3           0           0           0 
Missings: FALSE
Has weights: FALSE
Has blocking: FALSE
Has coordinates: FALSE
Classes: 3
Chemical   Normal    Overt 
      36       76       33 
Positive class: NA


Warning message in predict.WrappedModel(knnModel, newdata = diabetesTib):
“Provided data for prediction is not a pure data.frame but from class tbl_df, hence it will be converted.”

mmce        acc 
0.02758621 0.97241379

## Section 3.5

In [7]:
holdout <- makeResampleDesc(method = "Holdout", split = 2/3, stratify = TRUE)
holdoutCV <- resample(learner = knn, task = diabetesTask, 
                      resampling = holdout, measures = list(mmce, acc))

holdoutCV$aggr

Resampling: holdout
Measures:             mmce      acc       
[Resample] iter 1:    0.1020408 0.8979592 


Aggregated Result: mmce.test.mean=0.1020408,acc.test.mean=0.8979592




mmce.test.mean  acc.test.mean 
     0.1020408      0.8979592

## Exercise 2

In [8]:
holdout2 <- makeResampleDesc(method = "Holdout", split = 90/100, stratify = FALSE)
holdoutCV2 <- resample(learner = knn, task = diabetesTask,
                       resampling = holdout2, measures = list(mmce, acc))

holdoutCV2$aggr

Resampling: holdout
Measures:             mmce      acc       
[Resample] iter 1:    0.0666667 0.9333333 


Aggregated Result: mmce.test.mean=0.0666667,acc.test.mean=0.9333333




mmce.test.mean  acc.test.mean 
    0.06666667     0.93333333

## Section 3.5.1

In [9]:
calculateConfusionMatrix(holdoutCV$pred, relative = TRUE)
calculateConfusionMatrix(holdoutCV2$pred, relative = TRUE)

Relative confusion matrix (normalized by row/column):
          predicted
true       Chemical  Normal    Overt     -err.-   
  Chemical 0.75/0.82 0.17/0.07 0.08/0.10 0.25     
  Normal   0.00/0.00 1.00/0.93 0.00/0.00 0.00     
  Overt    0.18/0.18 0.00/0.00 0.82/0.90 0.18     
  -err.-        0.18      0.07      0.10 0.10     


Absolute confusion matrix:
          predicted
true       Chemical Normal Overt -err.-
  Chemical        9      2     1      3
  Normal          0     26     0      0
  Overt           2      0     9      2
  -err.-          2      2     1      5

Relative confusion matrix (normalized by row/column):
          predicted
true       Chemical  Normal    Overt     -err.-   
  Chemical 0.00/0.00 0.00/0.00 0.00/0.00 0.00     
  Normal   0.07/1.00 0.93/1.00 0.00/0.00 0.07     
  Overt    0.00/0.00 0.00/0.00 1.00/1.00 0.00     
  -err.-        1.00      0.00      0.00 0.07     


Absolute confusion matrix:
          predicted
true       Chemical Normal Overt -err.-
  Chemical        0      0     0      0
  Normal          1     13     0      1
  Overt           0      0     1      0
  -err.-          1      0     0      1

## Section 3.5.2

In [10]:
kFold <- makeResampleDesc(method = "RepCV", folds = 10, reps = 50, stratify = TRUE)
kFoldCV <- resample(learner = knn, task = diabetesTask, resampling = kFold, 
                    measures = list(mmce, acc))

kFoldCV$aggr

Resampling: repeated cross-validation
Measures:             mmce      acc       
[Resample] iter 1:    0.1333333 0.8666667 
[Resample] iter 2:    0.0714286 0.9285714 
[Resample] iter 3:    0.2000000 0.8000000 
[Resample] iter 4:    0.0666667 0.9333333 
[Resample] iter 5:    0.2142857 0.7857143 
[Resample] iter 6:    0.2000000 0.8000000 
[Resample] iter 7:    0.2142857 0.7857143 
[Resample] iter 8:    0.1428571 0.8571429 
[Resample] iter 9:    0.0769231 0.9230769 
[Resample] iter 10:   0.0625000 0.9375000 
[Resample] iter 11:   0.0714286 0.9285714 
[Resample] iter 12:   0.1875000 0.8125000 
[Resample] iter 13:   0.0000000 1.0000000 
[Resample] iter 14:   0.2142857 0.7857143 
[Resample] iter 15:   0.1333333 0.8666667 
[Resample] iter 16:   0.0769231 0.9230769 
[Resample] iter 17:   0.1250000 0.8750000 
[Resample] iter 18:   0.2142857 0.7857143 
[Resample] iter 19:   0.1333333 0.8666667 
[Resample] iter 20:   0.0000000 1.0000000 
[Resample] iter 21:   0.1333333 0.8666667 
[Resample] iter 

[Resample] iter 190:  0.1250000 0.8750000 
[Resample] iter 191:  0.0000000 1.0000000 
[Resample] iter 192:  0.1538462 0.8461538 
[Resample] iter 193:  0.2142857 0.7857143 
[Resample] iter 194:  0.0714286 0.9285714 
[Resample] iter 195:  0.0000000 1.0000000 
[Resample] iter 196:  0.0714286 0.9285714 
[Resample] iter 197:  0.0000000 1.0000000 
[Resample] iter 198:  0.0000000 1.0000000 
[Resample] iter 199:  0.1250000 0.8750000 
[Resample] iter 200:  0.1333333 0.8666667 
[Resample] iter 201:  0.0000000 1.0000000 
[Resample] iter 202:  0.1428571 0.8571429 
[Resample] iter 203:  0.1333333 0.8666667 
[Resample] iter 204:  0.0666667 0.9333333 
[Resample] iter 205:  0.3125000 0.6875000 
[Resample] iter 206:  0.0000000 1.0000000 
[Resample] iter 207:  0.0714286 0.9285714 
[Resample] iter 208:  0.1538462 0.8461538 
[Resample] iter 209:  0.0714286 0.9285714 
[Resample] iter 210:  0.0000000 1.0000000 
[Resample] iter 211:  0.0714286 0.9285714 
[Resample] iter 212:  0.0714286 0.9285714 
[Resample] 

[Resample] iter 381:  0.2000000 0.8000000 
[Resample] iter 382:  0.0666667 0.9333333 
[Resample] iter 383:  0.1250000 0.8750000 
[Resample] iter 384:  0.0000000 1.0000000 
[Resample] iter 385:  0.0666667 0.9333333 
[Resample] iter 386:  0.0000000 1.0000000 
[Resample] iter 387:  0.1333333 0.8666667 
[Resample] iter 388:  0.0000000 1.0000000 
[Resample] iter 389:  0.0714286 0.9285714 
[Resample] iter 390:  0.1428571 0.8571429 
[Resample] iter 391:  0.0769231 0.9230769 
[Resample] iter 392:  0.2000000 0.8000000 
[Resample] iter 393:  0.0714286 0.9285714 
[Resample] iter 394:  0.0000000 1.0000000 
[Resample] iter 395:  0.0666667 0.9333333 
[Resample] iter 396:  0.0714286 0.9285714 
[Resample] iter 397:  0.1333333 0.8666667 
[Resample] iter 398:  0.1428571 0.8571429 
[Resample] iter 399:  0.3125000 0.6875000 
[Resample] iter 400:  0.0000000 1.0000000 
[Resample] iter 401:  0.0666667 0.9333333 
[Resample] iter 402:  0.1333333 0.8666667 
[Resample] iter 403:  0.2142857 0.7857143 
[Resample] 

mmce.test.mean  acc.test.mean 
     0.1032049      0.8967951

## Exercise 3

In [11]:
kFoldResults <- tibble(run = integer(), Method = character(), Folds = integer(), 
                           Reps = integer(), mmce.test.mean = double(), acc.test.mean = double())

for (i in c(1, 2, 3, 4, 5)) {
  kFold2 <- makeResampleDesc(method = "RepCV", folds = 3, reps = 5, stratify = TRUE)
  kFoldCV2 <- resample(learner = knn, task = diabetesTask, resampling = kFold2, 
                    measures = list(mmce, acc))
  
  kFoldResults <- add_row(kFoldResults, run = i, Method = "RepCV", Folds = 3, Reps = 5, 
                 mmce.test.mean = kFoldCV2$aggr['mmce.test.mean'],
                 acc.test.mean = kFoldCV2$aggr['acc.test.mean'])

  kFold3 <- makeResampleDesc(method = "RepCV", folds = 3, reps = 500, stratify = TRUE)
  kFoldCV3 <- resample(learner = knn, task = diabetesTask, resampling = kFold3, 
                    measures = list(mmce, acc))

  kFoldResults <- add_row(kFoldResults, run = i, Method = "RepCV", Folds = 3, Reps = 500, 
                 mmce.test.mean = kFoldCV3$aggr['mmce.test.mean'],
                 acc.test.mean = kFoldCV3$aggr['acc.test.mean'])
}

tally <- kFoldResults %>% 
  group_by(Method, Folds, Reps) %>%
  summarize(mmce.mean = mean(mmce.test.mean),
            acc.mean = mean(acc.test.mean),
            count = n())

Resampling: repeated cross-validation
Measures:             mmce      acc       
[Resample] iter 1:    0.1458333 0.8541667 
[Resample] iter 2:    0.0416667 0.9583333 
[Resample] iter 3:    0.1836735 0.8163265 
[Resample] iter 4:    0.1041667 0.8958333 
[Resample] iter 5:    0.1250000 0.8750000 
[Resample] iter 6:    0.1224490 0.8775510 
[Resample] iter 7:    0.0833333 0.9166667 
[Resample] iter 8:    0.1250000 0.8750000 
[Resample] iter 9:    0.1632653 0.8367347 
[Resample] iter 10:   0.1041667 0.8958333 
[Resample] iter 11:   0.1224490 0.8775510 
[Resample] iter 12:   0.1458333 0.8541667 
[Resample] iter 13:   0.1020408 0.8979592 
[Resample] iter 14:   0.1041667 0.8958333 
[Resample] iter 15:   0.0833333 0.9166667 


Aggregated Result: mmce.test.mean=0.1170918,acc.test.mean=0.8829082


Resampling: repeated cross-validation
Measures:             mmce      acc       
[Resample] iter 1:    0.1428571 0.8571429 
[Resample] iter 2:    0.0833333 0.9166667 
[Resample] iter 3:    0.0208333 0.9

[Resample] iter 172:  0.1632653 0.8367347 
[Resample] iter 173:  0.1250000 0.8750000 
[Resample] iter 174:  0.0416667 0.9583333 
[Resample] iter 175:  0.0816327 0.9183673 
[Resample] iter 176:  0.1250000 0.8750000 
[Resample] iter 177:  0.0833333 0.9166667 
[Resample] iter 178:  0.1250000 0.8750000 
[Resample] iter 179:  0.1250000 0.8750000 
[Resample] iter 180:  0.1224490 0.8775510 
[Resample] iter 181:  0.1250000 0.8750000 
[Resample] iter 182:  0.1666667 0.8333333 
[Resample] iter 183:  0.1020408 0.8979592 
[Resample] iter 184:  0.1458333 0.8541667 
[Resample] iter 185:  0.1020408 0.8979592 
[Resample] iter 186:  0.1041667 0.8958333 
[Resample] iter 187:  0.1020408 0.8979592 
[Resample] iter 188:  0.1041667 0.8958333 
[Resample] iter 189:  0.1041667 0.8958333 
[Resample] iter 190:  0.1250000 0.8750000 
[Resample] iter 191:  0.0204082 0.9795918 
[Resample] iter 192:  0.0625000 0.9375000 
[Resample] iter 193:  0.1020408 0.8979592 
[Resample] iter 194:  0.1250000 0.8750000 
[Resample] 

[Resample] iter 363:  0.1224490 0.8775510 
[Resample] iter 364:  0.1020408 0.8979592 
[Resample] iter 365:  0.1458333 0.8541667 
[Resample] iter 366:  0.1041667 0.8958333 
[Resample] iter 367:  0.1250000 0.8750000 
[Resample] iter 368:  0.0625000 0.9375000 
[Resample] iter 369:  0.0816327 0.9183673 
[Resample] iter 370:  0.0833333 0.9166667 
[Resample] iter 371:  0.0612245 0.9387755 
[Resample] iter 372:  0.1041667 0.8958333 
[Resample] iter 373:  0.1041667 0.8958333 
[Resample] iter 374:  0.0816327 0.9183673 
[Resample] iter 375:  0.0833333 0.9166667 
[Resample] iter 376:  0.0833333 0.9166667 
[Resample] iter 377:  0.0816327 0.9183673 
[Resample] iter 378:  0.1666667 0.8333333 
[Resample] iter 379:  0.1458333 0.8541667 
[Resample] iter 380:  0.1020408 0.8979592 
[Resample] iter 381:  0.1250000 0.8750000 
[Resample] iter 382:  0.0833333 0.9166667 
[Resample] iter 383:  0.0625000 0.9375000 
[Resample] iter 384:  0.1020408 0.8979592 
[Resample] iter 385:  0.1020408 0.8979592 
[Resample] 

[Resample] iter 554:  0.1250000 0.8750000 
[Resample] iter 555:  0.0612245 0.9387755 
[Resample] iter 556:  0.1041667 0.8958333 
[Resample] iter 557:  0.1836735 0.8163265 
[Resample] iter 558:  0.1041667 0.8958333 
[Resample] iter 559:  0.1224490 0.8775510 
[Resample] iter 560:  0.1666667 0.8333333 
[Resample] iter 561:  0.0625000 0.9375000 
[Resample] iter 562:  0.0833333 0.9166667 
[Resample] iter 563:  0.1666667 0.8333333 
[Resample] iter 564:  0.0816327 0.9183673 
[Resample] iter 565:  0.0833333 0.9166667 
[Resample] iter 566:  0.1020408 0.8979592 
[Resample] iter 567:  0.1041667 0.8958333 
[Resample] iter 568:  0.1020408 0.8979592 
[Resample] iter 569:  0.0833333 0.9166667 
[Resample] iter 570:  0.1041667 0.8958333 
[Resample] iter 571:  0.1428571 0.8571429 
[Resample] iter 572:  0.0625000 0.9375000 
[Resample] iter 573:  0.1041667 0.8958333 
[Resample] iter 574:  0.0816327 0.9183673 
[Resample] iter 575:  0.0833333 0.9166667 
[Resample] iter 576:  0.1250000 0.8750000 
[Resample] 

[Resample] iter 745:  0.0625000 0.9375000 
[Resample] iter 746:  0.1020408 0.8979592 
[Resample] iter 747:  0.1041667 0.8958333 
[Resample] iter 748:  0.0833333 0.9166667 
[Resample] iter 749:  0.0612245 0.9387755 
[Resample] iter 750:  0.1250000 0.8750000 
[Resample] iter 751:  0.1041667 0.8958333 
[Resample] iter 752:  0.1250000 0.8750000 
[Resample] iter 753:  0.1224490 0.8775510 
[Resample] iter 754:  0.0833333 0.9166667 
[Resample] iter 755:  0.0833333 0.9166667 
[Resample] iter 756:  0.1224490 0.8775510 
[Resample] iter 757:  0.0416667 0.9583333 
[Resample] iter 758:  0.0833333 0.9166667 
[Resample] iter 759:  0.1224490 0.8775510 
[Resample] iter 760:  0.0816327 0.9183673 
[Resample] iter 761:  0.1458333 0.8541667 
[Resample] iter 762:  0.0625000 0.9375000 
[Resample] iter 763:  0.0625000 0.9375000 
[Resample] iter 764:  0.1041667 0.8958333 
[Resample] iter 765:  0.0816327 0.9183673 
[Resample] iter 766:  0.0833333 0.9166667 
[Resample] iter 767:  0.1020408 0.8979592 
[Resample] 

...

[Resample] iter 575:  0.0833333 0.9166667 
[Resample] iter 576:  0.1632653 0.8367347 
[Resample] iter 577:  0.1666667 0.8333333 
[Resample] iter 578:  0.1224490 0.8775510 
[Resample] iter 579:  0.1250000 0.8750000 
[Resample] iter 580:  0.1020408 0.8979592 
[Resample] iter 581:  0.1041667 0.8958333 
[Resample] iter 582:  0.1041667 0.8958333 
[Resample] iter 583:  0.1041667 0.8958333 
[Resample] iter 584:  0.0833333 0.9166667 
[Resample] iter 585:  0.0816327 0.9183673 
[Resample] iter 586:  0.0833333 0.9166667 
[Resample] iter 587:  0.0833333 0.9166667 
[Resample] iter 588:  0.1224490 0.8775510 
[Resample] iter 589:  0.0625000 0.9375000 
[Resample] iter 590:  0.0816327 0.9183673 
[Resample] iter 591:  0.1041667 0.8958333 
[Resample] iter 592:  0.1458333 0.8541667 
[Resample] iter 593:  0.0833333 0.9166667 
[Resample] iter 594:  0.1632653 0.8367347 
[Resample] iter 595:  0.1224490 0.8775510 
[Resample] iter 596:  0.1250000 0.8750000 
[Resample] iter 597:  0.0833333 0.9166667 
[Resample] 

[Resample] iter 766:  0.1458333 0.8541667 
[Resample] iter 767:  0.1020408 0.8979592 
[Resample] iter 768:  0.0416667 0.9583333 
[Resample] iter 769:  0.1666667 0.8333333 
[Resample] iter 770:  0.1250000 0.8750000 
[Resample] iter 771:  0.0816327 0.9183673 
[Resample] iter 772:  0.0612245 0.9387755 
[Resample] iter 773:  0.0833333 0.9166667 
[Resample] iter 774:  0.1458333 0.8541667 
[Resample] iter 775:  0.0833333 0.9166667 
[Resample] iter 776:  0.1836735 0.8163265 
[Resample] iter 777:  0.0833333 0.9166667 
[Resample] iter 778:  0.1041667 0.8958333 
[Resample] iter 779:  0.1020408 0.8979592 
[Resample] iter 780:  0.0833333 0.9166667 
[Resample] iter 781:  0.0833333 0.9166667 
[Resample] iter 782:  0.1250000 0.8750000 
[Resample] iter 783:  0.0408163 0.9591837 
[Resample] iter 784:  0.1428571 0.8571429 
[Resample] iter 785:  0.1250000 0.8750000 
[Resample] iter 786:  0.1041667 0.8958333 
[Resample] iter 787:  0.1020408 0.8979592 
[Resample] iter 788:  0.0833333 0.9166667 
[Resample] 

[Resample] iter 957:  0.1020408 0.8979592 
[Resample] iter 958:  0.1041667 0.8958333 
[Resample] iter 959:  0.1250000 0.8750000 
[Resample] iter 960:  0.0612245 0.9387755 
[Resample] iter 961:  0.0612245 0.9387755 
[Resample] iter 962:  0.0833333 0.9166667 
[Resample] iter 963:  0.0833333 0.9166667 
[Resample] iter 964:  0.1224490 0.8775510 
[Resample] iter 965:  0.1041667 0.8958333 
[Resample] iter 966:  0.0416667 0.9583333 
[Resample] iter 967:  0.0416667 0.9583333 
[Resample] iter 968:  0.0833333 0.9166667 
[Resample] iter 969:  0.1224490 0.8775510 
[Resample] iter 970:  0.1250000 0.8750000 
[Resample] iter 971:  0.1250000 0.8750000 
[Resample] iter 972:  0.1224490 0.8775510 
[Resample] iter 973:  0.0625000 0.9375000 
[Resample] iter 974:  0.1041667 0.8958333 
[Resample] iter 975:  0.1020408 0.8979592 
[Resample] iter 976:  0.1224490 0.8775510 
[Resample] iter 977:  0.0833333 0.9166667 
[Resample] iter 978:  0.1666667 0.8333333 
[Resample] iter 979:  0.1458333 0.8541667 
[Resample] 

[Resample] iter 1148: 0.1458333 0.8541667 
[Resample] iter 1149: 0.0612245 0.9387755 
[Resample] iter 1150: 0.1041667 0.8958333 
[Resample] iter 1151: 0.0833333 0.9166667 
[Resample] iter 1152: 0.1020408 0.8979592 
[Resample] iter 1153: 0.0833333 0.9166667 
[Resample] iter 1154: 0.1041667 0.8958333 
[Resample] iter 1155: 0.1020408 0.8979592 
[Resample] iter 1156: 0.1041667 0.8958333 
[Resample] iter 1157: 0.1224490 0.8775510 
[Resample] iter 1158: 0.1041667 0.8958333 
[Resample] iter 1159: 0.1458333 0.8541667 
[Resample] iter 1160: 0.1875000 0.8125000 
[Resample] iter 1161: 0.1224490 0.8775510 
[Resample] iter 1162: 0.1224490 0.8775510 
[Resample] iter 1163: 0.0625000 0.9375000 
[Resample] iter 1164: 0.0625000 0.9375000 
[Resample] iter 1165: 0.1250000 0.8750000 
[Resample] iter 1166: 0.1250000 0.8750000 
[Resample] iter 1167: 0.0816327 0.9183673 
[Resample] iter 1168: 0.1250000 0.8750000 
[Resample] iter 1169: 0.1458333 0.8541667 
[Resample] iter 1170: 0.1428571 0.8571429 
[Resample] 

[Resample] iter 1339: 0.1041667 0.8958333 
[Resample] iter 1340: 0.0625000 0.9375000 
[Resample] iter 1341: 0.1020408 0.8979592 
[Resample] iter 1342: 0.0833333 0.9166667 
[Resample] iter 1343: 0.1632653 0.8367347 
[Resample] iter 1344: 0.1041667 0.8958333 
[Resample] iter 1345: 0.1041667 0.8958333 
[Resample] iter 1346: 0.0816327 0.9183673 
[Resample] iter 1347: 0.1041667 0.8958333 
[Resample] iter 1348: 0.1458333 0.8541667 
[Resample] iter 1349: 0.1041667 0.8958333 
[Resample] iter 1350: 0.0816327 0.9183673 
[Resample] iter 1351: 0.1666667 0.8333333 
[Resample] iter 1352: 0.1250000 0.8750000 
[Resample] iter 1353: 0.0816327 0.9183673 
[Resample] iter 1354: 0.1041667 0.8958333 
[Resample] iter 1355: 0.1428571 0.8571429 
[Resample] iter 1356: 0.0833333 0.9166667 
[Resample] iter 1357: 0.0625000 0.9375000 
[Resample] iter 1358: 0.1224490 0.8775510 
[Resample] iter 1359: 0.1458333 0.8541667 
[Resample] iter 1360: 0.1041667 0.8958333 
[Resample] iter 1361: 0.1250000 0.8750000 
[Resample] 

In [12]:
tally

Method,Folds,Reps,mmce.mean,acc.mean,count
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
RepCV,3,5,0.1067347,0.8932653,5
RepCV,3,500,0.1046417,0.8953583,5


In [13]:
# folds = 10, reps = 50, stratify = TRUE
calculateConfusionMatrix(kFoldCV$pred, relative = TRUE)

# folds = 3, reps = 5, stratify = TRUE
calculateConfusionMatrix(kFoldCV2$pred, relative = TRUE)

# folds = 3, reps = 500, stratify = TRUE
calculateConfusionMatrix(kFoldCV3$pred, relative = TRUE)

Relative confusion matrix (normalized by row/column):
          predicted
true       Chemical  Normal    Overt     -err.-   
  Chemical 0.81/0.78 0.10/0.05 0.09/0.11 0.19     
  Normal   0.04/0.08 0.96/0.95 0.00/0.00 0.04     
  Overt    0.16/0.14 0.00/0.00 0.84/0.89 0.16     
  -err.-        0.22      0.05      0.11 0.10     


Absolute confusion matrix:
          predicted
true       Chemical Normal Overt -err.-
  Chemical     1455    175   170    345
  Normal        140   3660     0    140
  Overt         264      0  1386    264
  -err.-        404    175   170    749

Relative confusion matrix (normalized by row/column):
          predicted
true       Chemical  Normal    Overt     -err.-   
  Chemical 0.77/0.77 0.10/0.05 0.13/0.15 0.23     
  Normal   0.04/0.08 0.96/0.95 0.00/0.00 0.04     
  Overt    0.16/0.15 0.00/0.00 0.84/0.85 0.16     
  -err.-        0.23      0.05      0.15 0.11     


Absolute confusion matrix:
          predicted
true       Chemical Normal Overt -err.-
  Chemical      138     18    24     42
  Normal         14    366     0     14
  Overt          27      0   138     27
  -err.-         41     18    24     83

Relative confusion matrix (normalized by row/column):
          predicted
true       Chemical  Normal    Overt     -err.-   
  Chemical 0.79/0.79 0.12/0.05 0.09/0.10 0.21     
  Normal   0.04/0.08 0.96/0.95 0.00/0.00 0.04     
  Overt    0.15/0.13 0.00/0.00 0.85/0.90 0.15     
  -err.-        0.21      0.05      0.10 0.11     


Absolute confusion matrix:
          predicted
true       Chemical Normal Overt -err.-
  Chemical    14268   2098  1634   3732
  Normal       1468  36532     0   1468
  Overt        2415      0 14085   2415
  -err.-       3883   2098  1634   7615

## Exercise 4

In [14]:
# these methods are invalid, LOO does not "fold" nor does it "repeat"
# LOO2 <- makeResampleDesc(method = "LOO", stratify = TRUE)
# LOO3 <- makeResampleDesc(method = "LOO", reps = 5)

## Section 3.7

In [15]:
knnParamSpace <- makeParamSet(makeDiscreteParam("k", values = 1:10))
gridSearch <- makeTuneControlGrid()
cvForTuning <- makeResampleDesc("RepCV", folds = 10, reps = 20)
tunedK <- tuneParams("classif.knn", task = diabetesTask, 
                     resampling = cvForTuning,
                     par.set = knnParamSpace, control = gridSearch)

tunedK

[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def               Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5,6,7,8,9,10   -    TRUE     -
With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.1105952; time: 0.1 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.1064048; time: 0.1 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.0915476; time: 0.1 min
[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.0901190; time: 0.1 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0863095; time: 0.1 min
[Tune-x] 6: k=6
[Tune-y] 6: mmce.test.mean=0.0850238; time: 0.1 min
[Tune-x] 7: k=7
[Tune-y] 7: mmce.test.mean=0.0822857; time: 0.1 min
[Tune-x] 8: k=8
[Tune-y] 8: mmce.test.mean=0.0925952; time: 0.1 min
[Tune-x] 9: k=9
[Tune-y] 9: mmce.test.mean=0.0895476; time: 0.1 min
[Tune-x] 10: k=10
[Tune-y] 10: mmce.test.mean=0.0881429; time: 0.1 min
[Tune] Result: k=7 : mmce.test.mean=0.0822857


Tune result:
Op. pars: k=7
mmce.test.mean=0.0822857

In [16]:
knnTuningData <- generateHyperParsEffectData(tunedK)
plotHyperParsEffect(knnTuningData, x = "k", y = "mmce.test.mean", plot.type = "line") +
  theme_bw()

In [17]:
tunedKnn <- setHyperPars(makeLearner("classif.knn"), par.vals = tunedK$x)
tunedKnnModel <- train(tunedKnn, diabetesTask)

## Section 3.7.1 - hyperparameter tuning with nested cross-validation

In [18]:
inner <- makeResampleDesc("CV")
outer <- makeResampleDesc("RepCV", folds = 10, reps = 5)
knnWrapper <- makeTuneWrapper("classif.knn", resampling = inner,
                              par.set = knnParamSpace, control = gridSearch)
cvWithTuning <- resample(knnWrapper, diabetesTask, resampling = outer)
cvWithTuning

Resampling: repeated cross-validation
Measures:             mmce      
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def               Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5,6,7,8,9,10   -    TRUE     -
With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.1153846; time: 0.0 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.1000000; time: 0.0 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.0846154; time: 0.0 min
[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.1000000; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0923077; time: 0.0 min
[Tune-x] 6: k=6
[Tune-y] 6: mmce.test.mean=0.0769231; time: 0.0 min
[Tune-x] 7: k=7
[Tune-y] 7: mmce.test.mean=0.0846154; time: 0.0 min
[Tune-x] 8: k=8
[Tune-y] 8: mmce.test.mean=0.0692308; time: 0.0 min
[Tune-x] 9: k=9
[Tune-y] 9: mmce.test.mean=0.0923077; time: 0.0 min
[Tune-x] 10: k=10
[Tune-y] 10: mmce.test.mean=0.1000000; time: 0.0 min
[Tune] Result

With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.1153846; time: 0.0 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.1000000; time: 0.0 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.0923077; time: 0.0 min
[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.0923077; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0769231; time: 0.0 min
[Tune-x] 6: k=6
[Tune-y] 6: mmce.test.mean=0.0923077; time: 0.0 min
[Tune-x] 7: k=7
[Tune-y] 7: mmce.test.mean=0.1153846; time: 0.0 min
[Tune-x] 8: k=8
[Tune-y] 8: mmce.test.mean=0.1000000; time: 0.0 min
[Tune-x] 9: k=9
[Tune-y] 9: mmce.test.mean=0.1000000; time: 0.0 min
[Tune-x] 10: k=10
[Tune-y] 10: mmce.test.mean=0.0923077; time: 0.0 min
[Tune] Result: k=5 : mmce.test.mean=0.0769231
[Resample] iter 9:    0.1333333 
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def               Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5,6,7,8,9,10   -    TRUE     -
With 

[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.0758242; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0681319; time: 0.0 min
[Tune-x] 6: k=6
[Tune-y] 6: mmce.test.mean=0.0835165; time: 0.0 min
[Tune-x] 7: k=7
[Tune-y] 7: mmce.test.mean=0.0681319; time: 0.0 min
[Tune-x] 8: k=8
[Tune-y] 8: mmce.test.mean=0.0763736; time: 0.0 min
[Tune-x] 9: k=9
[Tune-y] 9: mmce.test.mean=0.0835165; time: 0.0 min
[Tune-x] 10: k=10
[Tune-y] 10: mmce.test.mean=0.0604396; time: 0.0 min
[Tune] Result: k=10 : mmce.test.mean=0.0604396
[Resample] iter 17:   0.2142857 
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def               Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5,6,7,8,9,10   -    TRUE     -
With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.1225275; time: 0.0 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.1302198; time: 0.0 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.1060440; time: 0.0 min
[Tun

[Tune-x] 8: k=8
[Tune-y] 8: mmce.test.mean=0.0846154; time: 0.0 min
[Tune-x] 9: k=9
[Tune-y] 9: mmce.test.mean=0.1076923; time: 0.0 min
[Tune-x] 10: k=10
[Tune-y] 10: mmce.test.mean=0.1076923; time: 0.0 min
[Tune] Result: k=8 : mmce.test.mean=0.0846154
[Resample] iter 25:   0.0000000 
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def               Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5,6,7,8,9,10   -    TRUE     -
With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.1219780; time: 0.0 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.1296703; time: 0.0 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.1142857; time: 0.0 min
[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.0912088; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0840659; time: 0.0 min
[Tune-x] 6: k=6
[Tune-y] 6: mmce.test.mean=0.0763736; time: 0.0 min
[Tune-x] 7: k=7
[Tune-y] 7: mmce.test.mean=0.0763736; time: 0.0 min
[Tune

[Resample] iter 33:   0.0000000 
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def               Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5,6,7,8,9,10   -    TRUE     -
With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.1065934; time: 0.0 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.0989011; time: 0.0 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.1076923; time: 0.0 min
[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.1153846; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0692308; time: 0.0 min
[Tune-x] 6: k=6
[Tune-y] 6: mmce.test.mean=0.0846154; time: 0.0 min
[Tune-x] 7: k=7
[Tune-y] 7: mmce.test.mean=0.0846154; time: 0.0 min
[Tune-x] 8: k=8
[Tune-y] 8: mmce.test.mean=0.0846154; time: 0.0 min
[Tune-x] 9: k=9
[Tune-y] 9: mmce.test.mean=0.0917582; time: 0.0 min
[Tune-x] 10: k=10
[Tune-y] 10: mmce.test.mean=0.0917582; time: 0.0 min
[Tune] Result: k=5 : mmce.test.mean=0.0692308
[Resa

Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.0983516; time: 0.0 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.1060440; time: 0.0 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.0835165; time: 0.0 min
[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.0835165; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0989011; time: 0.0 min
[Tune-x] 6: k=6
[Tune-y] 6: mmce.test.mean=0.0912088; time: 0.0 min
[Tune-x] 7: k=7
[Tune-y] 7: mmce.test.mean=0.0835165; time: 0.0 min
[Tune-x] 8: k=8
[Tune-y] 8: mmce.test.mean=0.0829670; time: 0.0 min
[Tune-x] 9: k=9
[Tune-y] 9: mmce.test.mean=0.0983516; time: 0.0 min
[Tune-x] 10: k=10
[Tune-y] 10: mmce.test.mean=0.0912088; time: 0.0 min
[Tune] Result: k=8 : mmce.test.mean=0.0829670
[Resample] iter 42:   0.1428571 
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def               Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5,6,7,8,9,10   -    TRUE     -
With control class: TuneControlGrid
Imput

[Tune-y] 4: mmce.test.mean=0.0769231; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0763736; time: 0.0 min
[Tune-x] 6: k=6
[Tune-y] 6: mmce.test.mean=0.0994505; time: 0.0 min
[Tune-x] 7: k=7
[Tune-y] 7: mmce.test.mean=0.0829670; time: 0.0 min
[Tune-x] 8: k=8
[Tune-y] 8: mmce.test.mean=0.0912088; time: 0.0 min
[Tune-x] 9: k=9
[Tune-y] 9: mmce.test.mean=0.0829670; time: 0.0 min
[Tune-x] 10: k=10
[Tune-y] 10: mmce.test.mean=0.0983516; time: 0.0 min
[Tune] Result: k=5 : mmce.test.mean=0.0763736
[Resample] iter 50:   0.0000000 


Aggregated Result: mmce.test.mean=0.0912381




Resample Result
Task: diabetesTib
Learner: classif.knn.tuned
Aggr perf: mmce.test.mean=0.0912381
Runtime: 185.542

## Section 3.7.2 - make predictions

In [19]:
newDiabetesPatients <- tibble(glucose = c(82, 108, 300),
                              insulin = c(361, 288, 1052),
                              sspg = c(200, 186, 135))
newDiabetesPatients
newPatentisPredictions <- predict(tunedKnnModel, newdata = newDiabetesPatients)
getPredictionResponse(newPatentisPredictions)

glucose,insulin,sspg
<dbl>,<dbl>,<dbl>
82,361,200
108,288,186
300,1052,135


Warning message in predict.WrappedModel(tunedKnnModel, newdata = newDiabetesPatients):
“Provided data for prediction is not a pure data.frame but from class tbl_df, hence it will be converted.”

[1] Normal Normal Overt 
Levels: Chemical Normal Overt

## Exercise 5

In [20]:
# load and summarize iris data
irisTib <- as_tibble(iris)
summary(irisTib)
head(irisTib, n = 5)

# visualize data
ggplot(irisTib, aes(Sepal.Length, Sepal.Width, col = Species)) + 
    geom_point() +
    theme_bw()

ggplot(irisTib, aes(Petal.Length, Petal.Width, col = Species)) + 
    geom_point() +
    theme_bw()

knnParamSpaceIris5 <- makeParamSet(makeDiscreteParam("k", values = 1:10))
irisTask5 <- makeClassifTask(data = irisTib, target = "Species")

gridSearchIris5 <- makeTuneControlGrid()
cvForTuningIris5 <- makeResampleDesc("RepCV", folds = 10, reps = 20)
tunedKIris5 <- tuneParams("classif.knn", task = irisTask5, 
                     resampling = cvForTuningIris5,
                     par.set = knnParamSpaceIris5, control = gridSearchIris5)

tunedKIris5
knnTuningDataIris5 <- generateHyperParsEffectData(tunedKIris5)
plotHyperParsEffect(knnTuningDataIris5, x = "k", y = "mmce.test.mean", plot.type = "line") +
  theme_bw()

  Sepal.Length    Sepal.Width     Petal.Length    Petal.Width   
 Min.   :4.300   Min.   :2.000   Min.   :1.000   Min.   :0.100  
 1st Qu.:5.100   1st Qu.:2.800   1st Qu.:1.600   1st Qu.:0.300  
 Median :5.800   Median :3.000   Median :4.350   Median :1.300  
 Mean   :5.843   Mean   :3.057   Mean   :3.758   Mean   :1.199  
 3rd Qu.:6.400   3rd Qu.:3.300   3rd Qu.:5.100   3rd Qu.:1.800  
 Max.   :7.900   Max.   :4.400   Max.   :6.900   Max.   :2.500  
       Species  
 setosa    :50  
 versicolor:50  
 virginica :50  
                
                
                

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa


Warning message in makeTask(type = type, data = data, weights = weights, blocking = blocking, :
“Provided data is not a pure data.frame but from class tbl_df, hence it will be converted.”[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def               Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5,6,7,8,9,10   -    TRUE     -
With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.0406667; time: 0.1 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.0483333; time: 0.1 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.0400000; time: 0.1 min
[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.0370000; time: 0.1 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0356667; time: 0.1 min
[Tune-x] 6: k=6
[Tune-y] 6: mmce.test.mean=0.0346667; time: 0.1 min
[Tune-x] 7: k=7
[Tune-y] 7: mmce.test.mean=0.0336667; time: 0.1 min
[Tune-x] 8: k=8
[Tune-y] 8: mmce.test.mean=0.0353333; time: 0.1 min
[Tune-x] 9: k=9
[Tune-y] 9: mmce.tes

Tune result:
Op. pars: k=7
mmce.test.mean=0.0336667

## Exercise 6

In [21]:
# set up inner and outer loops for hyperparameter tuning
irisTask6 <- makeClassifTask(data = irisTib, target = "Species")
knnParamSpaceIris6 <- makeParamSet(makeDiscreteParam("k", values = 1:5))
gridSearchIris6 <- makeTuneControlGrid()

innerIris6 <- makeResampleDesc("CV")
outerIris6 <- makeResampleDesc(method = "Holdout", split = 2/3, stratify = TRUE)
knnWrapperIris6 <- makeTuneWrapper("classif.knn", resampling = innerIris6,
                              par.set = knnParamSpaceIris6, control = gridSearchIris6)
cvIrisWithTuning6 <- resample(knnWrapperIris6, irisTask6, resampling = outerIris6)
cvIrisWithTuning6

Warning message in makeTask(type = type, data = data, weights = weights, blocking = blocking, :
“Provided data is not a pure data.frame but from class tbl_df, hence it will be converted.”Resampling: holdout
Measures:             mmce      
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def    Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5   -    TRUE     -
With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.0500000; time: 0.0 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.0700000; time: 0.0 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.0400000; time: 0.0 min
[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.0500000; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0400000; time: 0.0 min
[Tune] Result: k=5 : mmce.test.mean=0.0400000
[Resample] iter 1:    0.0392157 


Aggregated Result: mmce.test.mean=0.0392157




Resample Result
Task: irisTib
Learner: classif.knn.tuned
Aggr perf: mmce.test.mean=0.0392157
Runtime: 2.13606

## Exercise 7

In [22]:
# set up inner and outer loops for hyperparameter tuning
irisTask7 <- makeClassifTask(data = irisTib, target = "Species")
knnParamSpaceIris7 <- makeParamSet(makeDiscreteParam("k", values = 1:5))
gridSearchIris7 <- makeTuneControlGrid()

innerIris7 <- makeResampleDesc("CV")
outerIris7 <- makeResampleDesc("RepCV", folds = 5)
knnWrapperIris7 <- makeTuneWrapper("classif.knn", resampling = innerIris7,
                              par.set = knnParamSpaceIris7, control = gridSearchIris7)
cvIrisWithTuning7 <- resample(knnWrapperIris7, irisTask7, resampling = outerIris7)
cvIrisWithTuning7

Warning message in makeTask(type = type, data = data, weights = weights, blocking = blocking, :
“Provided data is not a pure data.frame but from class tbl_df, hence it will be converted.”Resampling: repeated cross-validation
Measures:             mmce      
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def    Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5   -    TRUE     -
With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.0583333; time: 0.0 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.0583333; time: 0.0 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.0583333; time: 0.0 min
[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.0500000; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0666667; time: 0.0 min
[Tune] Result: k=4 : mmce.test.mean=0.0500000
[Resample] iter 1:    0.0000000 
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def    Constr Req Tunable Trafo
k di

[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.0416667; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0416667; time: 0.0 min
[Tune] Result: k=3 : mmce.test.mean=0.0416667
[Resample] iter 13:   0.0000000 
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def    Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5   -    TRUE     -
With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.0250000; time: 0.0 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.0250000; time: 0.0 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.0166667; time: 0.0 min
[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.0166667; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0166667; time: 0.0 min
[Tune] Result: k=3 : mmce.test.mean=0.0166667
[Resample] iter 14:   0.1000000 
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def    Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5   -    TRUE     -


[Tune-y] 4: mmce.test.mean=0.0250000; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0250000; time: 0.0 min
[Tune] Result: k=1 : mmce.test.mean=0.0166667
[Resample] iter 26:   0.0666667 
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def    Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5   -    TRUE     -
With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.0583333; time: 0.0 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.0583333; time: 0.0 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.0500000; time: 0.0 min
[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.0583333; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0416667; time: 0.0 min
[Tune] Result: k=5 : mmce.test.mean=0.0416667
[Resample] iter 27:   0.0000000 
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def    Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5   -    TRUE     -
With control cla

[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0250000; time: 0.0 min
[Tune] Result: k=4 : mmce.test.mean=0.0166667
[Resample] iter 39:   0.0666667 
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def    Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5   -    TRUE     -
With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: k=1
[Tune-y] 1: mmce.test.mean=0.0500000; time: 0.0 min
[Tune-x] 2: k=2
[Tune-y] 2: mmce.test.mean=0.0666667; time: 0.0 min
[Tune-x] 3: k=3
[Tune-y] 3: mmce.test.mean=0.0500000; time: 0.0 min
[Tune-x] 4: k=4
[Tune-y] 4: mmce.test.mean=0.0416667; time: 0.0 min
[Tune-x] 5: k=5
[Tune-y] 5: mmce.test.mean=0.0416667; time: 0.0 min
[Tune] Result: k=5 : mmce.test.mean=0.0416667
[Resample] iter 40:   0.0000000 
[Tune] Started tuning learner classif.knn for parameter set:
      Type len Def    Constr Req Tunable Trafo
k discrete   -   - 1,2,3,4,5   -    TRUE     -
With control class: TuneControlGrid
Imputation value: 1
[Tune-x] 1: 

Resample Result
Task: irisTib
Learner: classif.knn.tuned
Aggr perf: mmce.test.mean=0.0473333
Runtime: 97.4589